In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

message_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into one of the following two types:
            - Product inquiry, order history inquiry, order change history inquiry, order cancellation history inquiry: '문의'
            - Order request, order change request, order cancellation request: '요청'
            
            You need to review the messages in the Messages Placeholder from the latest to the oldest.

            Consider the previous AI responses and their classifications to understand the intent behind the current input. 
            Use this context to make an accurate classification. 
            If the latest AI response was classified as '요청', and the current input is related to an order, it is likely a '요청'.
            
            Additionally, if the input contains order details, it should be classified as '요청'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [4]:
classify_message_chain = message_type_prompt | model

In [6]:
store = {}

In [7]:
from typing import List, Optional
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    save_mode: Optional[str] = Field(default="both")  # "input", "output", "both"
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """조건에 따라 메시지를 저장"""
        if self.save_mode == "input":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            self.messages.extend(input_messages)
        elif self.save_mode == "output":
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            self.messages.extend(output_messages)
        elif self.save_mode == "both":
            self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

# store = {}
def get_by_session_id(session_id: str, save_mode: str = "both") -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory(save_mode=save_mode)
    return store[session_id]

def create_chain_with_memory(chain, save_mode: str) -> RunnableWithMessageHistory:
    def get_session_history_with_mode(session_id: str) -> BaseChatMessageHistory:
        return get_by_session_id(session_id, save_mode)
    
    return RunnableWithMessageHistory(
        chain,
        get_session_history=get_session_history_with_mode,
        input_messages_key="input",
        history_messages_key="chat_history",
    )

In [8]:
chain_with_history_input_only = create_chain_with_memory(classify_message_chain, "input")
chain_with_history_input_only

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into one of the following two types:\n            - Product inquiry, order history inquiry, order change history inquiry, order cancellation history inquiry: '문의'\n            - Order 

In [9]:
response_input_only = chain_with_history_input_only.invoke(
    {"input": "주문 변경하고 싶어"},
    config={"configurable": {"session_id": "input_only"}}
)

Parent run ebf00f24-7d41-4cbe-b823-c0b68d3913f8 not found for run ec5063ba-1a7c-4a24-819e-b59ebf89f3a6. Treating as a root run.


In [10]:
store

{'input_only': InMemoryHistory(messages=[HumanMessage(content='주문 변경하고 싶어')], save_mode='input')}

# context 추가 가능

In [11]:
from typing import List, Optional
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    save_mode: Optional[str] = Field(default="both")  # "input", "output", "both"
    
    def add_messages(self, messages: List[BaseMessage], context: Optional[str] = None) -> None:
        """조건에 따라 메시지를 저장하고, 맥락을 결합"""
        if self.save_mode == "input":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            self.messages.extend(input_messages)
        elif self.save_mode == "output":
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            if context and output_messages:
                output_messages[0].content = f"{context}\n{output_messages[0].content}"
            self.messages.extend(output_messages)
        elif self.save_mode == "both":
            if context:
                first_output_added = False
                for msg in messages:
                    if isinstance(msg, AIMessage) and not first_output_added:
                        msg.content = f"{context}\n{msg.content}"
                        first_output_added = True
            self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

store = {}

def get_by_session_id(session_id: str, save_mode: str = "both") -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory(save_mode=save_mode)
    return store[session_id]


In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_models import ChatAnthropic
from langchain_core.runnables.history import RunnableWithMessageHistory

def create_chain_with_memory(chain, save_mode: str) -> RunnableWithMessageHistory:
    def get_session_history_with_mode(session_id: str) -> BaseChatMessageHistory:
        return get_by_session_id(session_id, save_mode)
    
    class CustomRunnableWithMessageHistory(RunnableWithMessageHistory):
        def invoke(self, input: dict, config: Optional[dict] = None, context: Optional[str] = None) -> dict:
            history = self.get_session_history(config["configurable"]["session_id"])
            result = super().invoke(input, config)
            history.add_messages([result], context=context)
            return result

    return CustomRunnableWithMessageHistory(
        chain,
        get_session_history=get_session_history_with_mode,
        input_messages_key="question",
        history_messages_key="history",
    )


In [14]:
# 예제 사용법
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = prompt | model

In [25]:
store = {}

In [26]:
# 입력 메시지만 저장하는 체인 생성
chain_with_history_input_only = create_chain_with_memory(chain, "input")

# 출력 메시지만 저장하는 체인 생성
chain_with_history_output_only = create_chain_with_memory(chain, "output")

# 입력 및 출력 메시지 모두 저장하는 체인 생성
chain_with_history_both = create_chain_with_memory(chain, "both")

# 테스트 실행
response_input_only = chain_with_history_input_only.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "input_only"}},
    context="Additional context for input only"
)
print(response_input_only)

response_output_only = chain_with_history_output_only.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "output_only"}},
    context="Additional context for output only"
)
print(response_output_only)

response_both = chain_with_history_both.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "both"}},
    context="Additional context for both"
)
print(response_both)

# 저장된 메시지 확인
print(store)

Parent run fad7cfeb-31ca-4eae-87e9-8ad83be9f8ec not found for run de16315f-4de4-4ef4-838e-d235b61b5290. Treating as a root run.
Parent run 9a768410-253b-4326-aef2-637ca5961fb4 not found for run 32cef166-6c93-424d-95c8-e2ba3095378b. Treating as a root run.


content='Cosine is a trigonometric function that represents the ratio of the length of the adjacent side to the length of the hypotenuse in a right triangle. In simpler terms, cosine is a function that relates the angle of a right triangle to the ratio of the length of the adjacent side to the hypotenuse. It is commonly denoted as cos.' response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 25, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-c33c37f3-f178-47c4-973c-c1fcd934aa36-0'


Parent run 2b927dfe-9b46-4682-a467-bcda1b7bafa2 not found for run c2ec2851-7b56-45ad-a569-f257f8752347. Treating as a root run.


content='Additional context for output only\nCosine is a trigonometric function that represents the ratio of the length of the adjacent side to the length of the hypotenuse in a right triangle. In simpler terms, cosine is a function that gives the ratio of the length of the side adjacent to an angle in a right triangle to the length of the hypotenuse. It is denoted by cos(theta) or cos(x), where theta or x is the angle in the triangle.' response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 25, 'total_tokens': 115}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-8eef939b-067d-4379-968f-dfda42ebff77-0'
content='Additional context for both\nCosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle. In simpler terms, it is a mathematical function that describes the relationship between the length of the adjacent side and the hypotenuse in a rig

In [29]:
store["input_only"].messages

[HumanMessage(content='What does cosine mean?')]

In [27]:
store["output_only"].messages

[AIMessage(content='Additional context for output only\nCosine is a trigonometric function that represents the ratio of the length of the adjacent side to the length of the hypotenuse in a right triangle. In simpler terms, cosine is a function that gives the ratio of the length of the side adjacent to an angle in a right triangle to the length of the hypotenuse. It is denoted by cos(theta) or cos(x), where theta or x is the angle in the triangle.', response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 25, 'total_tokens': 115}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8eef939b-067d-4379-968f-dfda42ebff77-0'),
 AIMessage(content='Additional context for output only\nCosine is a trigonometric function that represents the ratio of the length of the adjacent side to the length of the hypotenuse in a right triangle. In simpler terms, cosine is a function that gives the ratio of the length of the side 

In [28]:
store["both"].messages

[HumanMessage(content='What does cosine mean?'),
 AIMessage(content='Additional context for both\nCosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle. In simpler terms, it is a mathematical function that describes the relationship between the length of the adjacent side and the hypotenuse in a right triangle. The cosine function is commonly used in mathematics, physics, engineering, and various other fields to solve problems involving angles and distances.', response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 25, 'total_tokens': 106}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d69cd060-ddc7-4757-bd01-1b7e67e1d5c3-0'),
 AIMessage(content='Additional context for both\nCosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle. In simpler terms, it is a mathematical func

---

In [30]:
from typing import List, Optional
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    save_mode: Optional[str] = Field(default="both")  # "input", "output", "both"
    
    def add_messages(self, messages: List[BaseMessage], context: Optional[str] = None) -> None:
        """조건에 따라 메시지를 저장하고, 맥락을 결합"""
        new_messages = []
        if self.save_mode == "input":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            new_messages.extend(input_messages)
        elif self.save_mode == "output":
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            if context and output_messages:
                output_messages[0].content = f"{context}\n{output_messages[0].content}"
            new_messages.extend(output_messages)
        elif self.save_mode == "both":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            if context and output_messages:
                output_messages[0].content = f"{context}\n{output_messages[0].content}"
            new_messages.extend(input_messages + output_messages)
        
        # 메시지 중복 저장 방지
        self.messages.extend(new_messages)

    def clear(self) -> None:
        self.messages = []

store = {}

def get_by_session_id(session_id: str, save_mode: str = "both") -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory(save_mode=save_mode)
    return store[session_id]


In [31]:
# 입력 메시지만 저장하는 체인 생성
chain_with_history_input_only = create_chain_with_memory(chain, "input")

# 출력 메시지만 저장하는 체인 생성
chain_with_history_output_only = create_chain_with_memory(chain, "output")

# 입력 및 출력 메시지 모두 저장하는 체인 생성
chain_with_history_both = create_chain_with_memory(chain, "both")

# 테스트 실행
response_input_only = chain_with_history_input_only.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "input_only"}},
    context="Additional context for input only"
)
print(response_input_only)

response_output_only = chain_with_history_output_only.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "output_only"}},
    context="Additional context for output only"
)
print(response_output_only)

response_both = chain_with_history_both.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "both"}},
    context="Additional context for both"
)
print(response_both)

# 저장된 메시지 확인
print(store)

Parent run 753322d6-b6cb-4bff-af74-5caef7351119 not found for run a6c3cfcc-8faf-4e41-b1b2-94c993fa770c. Treating as a root run.
Parent run ca9363f9-ddb0-44e9-a222-de043c6cebce not found for run 2c38c880-e72a-4df2-b472-784754fd4c5b. Treating as a root run.


content='Cosine is a trigonometric function that represents the ratio of the adjacent side of a right triangle to the hypotenuse. In a right triangle, the cosine of an angle is calculated by dividing the length of the side adjacent to the angle by the length of the hypotenuse. The cosine function is commonly denoted as cos(theta) or cos(x), where theta or x represents the angle in radians.' response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 25, 'total_tokens': 108}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-ebb4b4d2-0dd6-4346-867d-8d3b085686c4-0'


Parent run 855c32e4-63f9-4bba-bf97-9b507eff4a66 not found for run 27e1bd4d-b83e-44e8-b8c1-3f69e5319754. Treating as a root run.


content='Additional context for output only\nCosine is a trigonometric function that describes the ratio of the length of the adjacent side of a right triangle to the length of the hypotenuse. In simpler terms, cosine is a function that gives the ratio of the adjacent side length to the hypotenuse in a right triangle. It is often denoted as cos.' response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 25, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4b9b1087-846c-453a-8ac3-6a0dc5cb9a94-0'
content='Additional context for both\nCosine is a mathematical function that represents the ratio of the length of the adjacent side to the length of the hypotenuse in a right triangle. In simpler terms, cosine is a trigonometric function that relates the angle of a right triangle to the ratio of the lengths of two sides of the triangle. It is commonly denoted as cos.' response_metadata={'token_u

In [35]:
store["input_only"].messages

[HumanMessage(content='What does cosine mean?')]

In [36]:
store["output_only"].messages

[AIMessage(content='Additional context for output only\nCosine is a trigonometric function that describes the ratio of the length of the adjacent side of a right triangle to the length of the hypotenuse. In simpler terms, cosine is a function that gives the ratio of the adjacent side length to the hypotenuse in a right triangle. It is often denoted as cos.', response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 25, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4b9b1087-846c-453a-8ac3-6a0dc5cb9a94-0'),
 AIMessage(content='Additional context for output only\nCosine is a trigonometric function that describes the ratio of the length of the adjacent side of a right triangle to the length of the hypotenuse. In simpler terms, cosine is a function that gives the ratio of the adjacent side length to the hypotenuse in a right triangle. It is often denoted as cos.', response_metadata={'tok

In [37]:
store["both"].messages

[HumanMessage(content='What does cosine mean?'),
 AIMessage(content='Additional context for both\nCosine is a mathematical function that represents the ratio of the length of the adjacent side to the length of the hypotenuse in a right triangle. In simpler terms, cosine is a trigonometric function that relates the angle of a right triangle to the ratio of the lengths of two sides of the triangle. It is commonly denoted as cos.', response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 25, 'total_tokens': 95}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f30efc64-c199-4923-bc1e-1116b41dc29d-0'),
 AIMessage(content='Additional context for both\nCosine is a mathematical function that represents the ratio of the length of the adjacent side to the length of the hypotenuse in a right triangle. In simpler terms, cosine is a trigonometric function that relates the angle of a right triangle to the ratio of the

In [57]:
store = {}
store

{}

In [58]:
from typing import List, Optional
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    save_mode: Optional[str] = Field(default="both")  # "input", "output", "both"
    
    def add_messages(self, messages: List[BaseMessage], context: Optional[str] = None) -> None:
        """조건에 따라 메시지를 저장하고, 맥락을 결합"""
        new_messages = []
        if self.save_mode == "input":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            new_messages.extend(input_messages)
        elif self.save_mode == "output":
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            if context and output_messages:
                output_messages[0].content = f"{context}\n{output_messages[0].content}"
            new_messages.extend(output_messages)
        elif self.save_mode == "both":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            if context and output_messages:
                output_messages[0].content = f"{context}\n{output_messages[0].content}"
            new_messages.extend(input_messages + output_messages)
        
        # 메시지 중복 저장 방지
        self.messages.extend(new_messages)

    def clear(self) -> None:
        self.messages = []

store = {}

def get_by_session_id(session_id: str, save_mode: str = "both") -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory(save_mode=save_mode)
    return store[session_id]

In [59]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_models import ChatAnthropic
from langchain_core.runnables.history import RunnableWithMessageHistory

def create_chain_with_memory(chain, save_mode: str) -> RunnableWithMessageHistory:
    def get_session_history_with_mode(session_id: str) -> BaseChatMessageHistory:
        return get_by_session_id(session_id, save_mode)
    
    class CustomRunnableWithMessageHistory(RunnableWithMessageHistory):
        def invoke(self, input: dict, config: Optional[dict] = None, context: Optional[str] = None) -> dict:
            history = self.get_session_history(config["configurable"]["session_id"])
            result = super().invoke(input, config)
            history.add_messages([result], context=context)
            return result

    return CustomRunnableWithMessageHistory(
        chain,
        get_session_history=get_session_history_with_mode,
        input_messages_key="question",
        history_messages_key="history",
    )

In [60]:
response_output_only = chain_with_history_output_only.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "output_only"}},
    context="Additional context for output only"
)
print(response_output_only)

Parent run ba33c42d-5999-4650-8d78-f0d0e97f79a4 not found for run 4b976ff9-729c-4b2b-b42c-4d23e845f9e0. Treating as a root run.


content='Additional context for output only\nCosine is a mathematical function that relates the angle of a right triangle to the ratio of the length of the adjacent side to the length of the hypotenuse. In simpler terms, cosine is a trigonometric function that gives the ratio of the adjacent side to the hypotenuse of a right triangle for a given angle. It is denoted by the abbreviation cos.' response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 25, 'total_tokens': 100}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-0fb16627-5895-46d3-99ea-135896f8e44a-0'


In [61]:
store["output_only"].messages

[AIMessage(content='Additional context for output only\nCosine is a mathematical function that relates the angle of a right triangle to the ratio of the length of the adjacent side to the length of the hypotenuse. In simpler terms, cosine is a trigonometric function that gives the ratio of the adjacent side to the hypotenuse of a right triangle for a given angle. It is denoted by the abbreviation cos.', response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 25, 'total_tokens': 100}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0fb16627-5895-46d3-99ea-135896f8e44a-0'),
 AIMessage(content='Additional context for output only\nCosine is a mathematical function that relates the angle of a right triangle to the ratio of the length of the adjacent side to the length of the hypotenuse. In simpler terms, cosine is a trigonometric function that gives the ratio of the adjacent side to the hypotenuse of a right

---

In [62]:
from typing import List, Optional
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    save_mode: Optional[str] = Field(default="both")  # "input", "output", "both"
    
    def add_messages(self, messages: List[BaseMessage], context: Optional[str] = None) -> None:
        """조건에 따라 메시지를 저장하고, 맥락을 결합"""
        new_messages = []
        if self.save_mode == "input":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            new_messages.extend(input_messages)
        elif self.save_mode == "output":
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            if context and output_messages:
                output_messages[0].content = f"{context}\n{output_messages[0].content}"
            new_messages.extend(output_messages)
        elif self.save_mode == "both":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            if context and output_messages:
                output_messages[0].content = f"{context}\n{output_messages[0].content}"
            new_messages.extend(input_messages + output_messages)
        
        # 메시지 중복 저장 방지
        self.messages.extend(new_messages)

    def clear(self) -> None:
        self.messages = []

store = {}

def get_by_session_id(session_id: str, save_mode: str = "both") -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory(save_mode=save_mode)
    return store[session_id]


In [63]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_models import ChatAnthropic
from langchain_core.runnables.history import RunnableWithMessageHistory

def create_chain_with_memory(chain, save_mode: str) -> RunnableWithMessageHistory:
    def get_session_history_with_mode(session_id: str) -> BaseChatMessageHistory:
        return get_by_session_id(session_id, save_mode)
    
    class CustomRunnableWithMessageHistory(RunnableWithMessageHistory):
        def invoke(self, input: dict, config: Optional[dict] = None, context: Optional[str] = None) -> dict:
            history = self.get_session_history(config["configurable"]["session_id"])
            if self.history_messages_key:
                history_messages = history.messages
                input[self.history_messages_key] = history_messages

            result = super().invoke(input, config)
            result_messages = result.get('messages') if isinstance(result, dict) else [result]
            history.add_messages(result_messages, context=context)
            return result

    return CustomRunnableWithMessageHistory(
        chain,
        get_session_history=get_session_history_with_mode,
        input_messages_key="question",
        history_messages_key="history",
    )


In [65]:
# 예제 사용법
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = prompt | model

# 입력 메시지만 저장하는 체인 생성
chain_with_history_input_only = create_chain_with_memory(chain, "input")

# 출력 메시지만 저장하는 체인 생성
chain_with_history_output_only = create_chain_with_memory(chain, "output")

# 입력 및 출력 메시지 모두 저장하는 체인 생성
chain_with_history_both = create_chain_with_memory(chain, "both")

# 테스트 실행
response_input_only = chain_with_history_input_only.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "input_only"}},
    context="Additional context for input only"
)
print(response_input_only)

response_output_only = chain_with_history_output_only.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "output_only"}},
    context="Additional context for output only"
)
print(response_output_only)

response_both = chain_with_history_both.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "both"}},
    context="Additional context for both"
)
print(response_both)

# 저장된 메시지 확인
print(store)


Parent run 8e2b7687-6dd3-4c4d-a9be-7b9ed24b7b6c not found for run ced38e5e-101e-40b4-b178-9961fd1a758f. Treating as a root run.
Parent run b3c2d94c-f837-41fc-8452-cbf9163fe40e not found for run 660f49b4-5834-4e72-9ccc-24ef6e662363. Treating as a root run.


content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right-angled triangle. In simpler terms, it is a mathematical function that helps to relate the angles of a triangle to the lengths of its sides. The cosine of an angle is calculated by dividing the length of the adjacent side by the length of the hypotenuse.' response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 25, 'total_tokens': 102}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-1fd4aa67-bb8c-4e73-b262-8ea25d7888a7-0'


Parent run 1a685087-f93f-4aa6-8cc1-65c057023468 not found for run ed77101a-c36c-4aa6-98e7-5e5e7a0925ec. Treating as a root run.


content="Additional context for output only\nCosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle. In simpler terms, it's a function that relates the angle of a right triangle to the ratio of the length of the adjacent side to the length of the hypotenuse. The cosine function is denoted as cos(theta), where theta is the angle in the triangle." response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 25, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-517cf84f-5719-4978-94c6-993519a339db-0'
content='Additional context for both\nCosine is a trigonometric function that relates the length of the adjacent side of a right triangle to the length of the hypotenuse. In a right triangle, the cosine of an angle is equal to the length of the side adjacent to that angle divided by the length of the hypotenuse. It is often denoted 

In [68]:
store["output_only"].messages

[AIMessage(content="Additional context for output only\nCosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle. In simpler terms, it's a function that relates the angle of a right triangle to the ratio of the length of the adjacent side to the length of the hypotenuse. The cosine function is denoted as cos(theta), where theta is the angle in the triangle.", response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 25, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-517cf84f-5719-4978-94c6-993519a339db-0'),
 AIMessage(content="Additional context for output only\nCosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle. In simpler terms, it's a function that relates the angle of a right triangle to the ratio of the length of the adjacent side to the length of 